# Analyze predicted escape from the 3K antibodies

## Import `Python` modules

In [2]:
import os
import glob
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Read in data that predicts how much each virus has escaped each antibody

In [3]:
f = '/fh/fast/matsen_e/shared/dmsa-pred/ncov-dmsa/results/cao_predictions/imprinted_2022_abs/all_predictions.csv'
escape_df = pd.read_csv(f)
escape_df.head()

,strain,aa_substitutions,escape_score,antibody,group,date,Nextstrain_clade,time
0,ON641975.1,NaN,0.000000,BD55-6049,E3,2022-01-08,?,2.038356
1,ON641975.1,G446S N501Y,0.020197,BD56-965,B,2022-01-08,?,2.038356
2,ON641975.1,NaN,0.000000,XGv-318,F1,2022-01-08,?,2.038356
3,ON641975.1,NaN,0.000000,BD55-5687,E3,2022-01-08,?,2.038356
4,ON641975.1,E484A,0.005702,XGv-214,E2.1,2022-01-08,?,2.038356


## Get a feel for the data

How many antibodies are in the dataset?

How many viruses are in the dataset?

How many different antibody groups are there?

## Analyze escape

* How many antibodies have been escaped? To answer this, we can plot the average escape score of all viruses from the past 6 months
* How has average escape changed over time? To answer this, we can plot average escape over time. This is a lot of curves to plot. We've come up with a few ideas for 
    * making ~30 plots each with 100 curves
    * plotting data separately for different groups
    * 
* Can we group antibodies by their escape patterns?